In [9]:
!pip install gurobipy
!pip install torch_geometric
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


import numpy as np
import gurobipy as gp
from gurobipy import GRB
import torch
from sklearn.model_selection import train_test_split

import itertools
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances as sklearn_pairwise_distances
import networkx as nx
 # Use "cosine" for cosine similarity
import heapq


os.environ["GRB_LICENSE_FILE"] = "gurobi (3).lic"
os.environ["GRB_WLSACCESSID"] = "f218200d-1f8d-4342-83f5-b7b2d9263751"  # Replace with your actual WLSACCESSID
os.environ["GRB_WLSSECRET"] = "528d596b-babc-4a1e-bda2-693c44f4f006"  # Replace with your actual WLSSECRET
os.environ["GRB_LICENSEID"] = "840285"  # Replace with your actual LICENSEID

from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from torch_geometric.datasets import WebKB, WikipediaNetwork, Actor

from torch_geometric.datasets import FacebookPagePage

from torch_geometric.datasets import SNAPDataset

import numpy as np
import torch
import random
import networkx as nx
from torch_geometric.datasets import WebKB
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
from sortedcontainers import SortedSet  # Ordered set
from torch_geometric.utils import to_networkx

from torch_geometric.datasets import WikipediaNetwork

In [56]:


# 📌 Load the Cornell Dataset
dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
data = dataset[0]
data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)

# ✅ Remove duplicate edges
data.edge_index = torch.unique(data.edge_index, dim=1)

# ✅ Convert to NetworkX Graph to Find Components
G = to_networkx(data, to_undirected=True)
components = list(nx.connected_components(G))
num_components = len(components)

print(f"\n✅ Loaded  dataset with {data.num_nodes} nodes, {data.edge_index.shape[1]} edges")
print(f"   - Features: {data.x.shape[1]}")
print(f"   - Number of Classes: {len(set(data.y.numpy()))}")
print(f"   - Number of Components: {num_components}")

# ✅ Set Training Percentage
train_percentage = 0.65

# ✅ Select Training Nodes
train_indices = []
for component in components:
    component = list(component)
    t = max(1, int(train_percentage * len(component)))
    sampled_nodes = np.random.choice(component, t, replace=False)
    train_indices.extend(sampled_nodes)

np.random.shuffle(train_indices)

# ✅ Initialize Prediction Labels
y_pred = np.full(data.num_nodes, -1)
for idx in train_indices:
    y_pred[idx] = data.y[idx].item()

# ✅ Compute Label Distribution
total_label_counts = Counter(data.y.numpy())
num_labels = len(set(data.y.numpy()))
alpha = 1  # Laplace smoothing

dataset_label_distribution = {
    lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
    for lbl in total_label_counts.keys()
}

train_label_counts = Counter(y_pred[train_indices])

train_label_distribution = {
    lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
    for lbl in dataset_label_distribution.keys()
}

# ✅ Precompute Neighbors for Each Node
A = torch.zeros((data.num_nodes, data.num_nodes))
A[data.edge_index[0], data.edge_index[1]] = 1
A = A.numpy()
neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

# ✅ Track Instances Assigned to Each Label
class_instances = defaultdict(set)
for idx in train_indices:
    class_instances[data.y[idx].item()].add(idx)

# ✅ Ordered Set for Managing Unlabeled Vertices
ordered_set = SortedSet()
train_labeled_nodes = set(train_indices)

for node in range(data.num_nodes):
    if y_pred[node] == -1:  # Only process unlabeled nodes
        labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
        num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
        num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
        total_neighbors = len(neighbors_dict[node])

        if total_neighbors > 0:
            weighted_score = (num_labeled_by_propagation + 3 * num_labeled_by_train) / total_neighbors
        else:
            weighted_score = 0

        ordered_set.add((-weighted_score, node))

# ✅ Iteratively Label the Most Constrained Nodes
while ordered_set:
    _, node = ordered_set.pop(0)

    if y_pred[node] != -1:
        continue

    # Get labeled neighbors
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    # 🚨 If this node has 0 labeled neighbors
    if not neighbor_labels:
        print(f"\n🚨 STRANGE: Node {node} was chosen, but it has 0 labeled neighbors!")
        break

    # Compute Neighbor Label Distribution
    neighbor_label_counts = Counter(neighbor_labels)
    total_labeled_neighbors = len(neighbor_labels)

    if total_labeled_neighbors > 0:
        neighbor_label_distribution = {
            lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
        }
    else:
        neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

    # ✅ Compute Feature Distance to Each Class
    # feature_diffs = {}
    # for lbl in dataset_label_distribution.keys():
    #   if class_instances[lbl]:  # Only compute if we have instances of the label
    #       instance_features = data.x[list(class_instances[lbl])]
    #       node_feature = data.x[node].unsqueeze(0)

    #       # Compute distances for each instance
    #       distances = torch.norm(instance_features - node_feature, dim=1).tolist()

    #       # Apply weight based on whether the instance is from training or predicted labels
    #       weighted_distances = [
    #           (3 * dist) if instance in train_indices else dist  # Give 3x weight if in training set
    #           for instance, dist in zip(class_instances[lbl], distances)
    #       ]

    #       # Compute weighted average distance
    #       avg_distance = sum(weighted_distances) / len(weighted_distances)
    #       feature_diffs[lbl] = avg_distance
    #   else:
    #       feature_diffs[lbl] = 0  # No penalty if no instances exist

    feature_diffs = {}
    for lbl in dataset_label_distribution.keys():
        if class_instances[lbl]:  # Only compute if we have instances of the label
            instance_features = data.x[list(class_instances[lbl])]
            node_feature = data.x[node].unsqueeze(0)
            print(len(instance_features),len(node_feature))
            avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
            feature_diffs[lbl] = avg_distance
        else:
            feature_diffs[lbl] = 0  # No penalty if no instances exist

    # ✅ Select the Best Label Based on the Adjusted Score
    best_label_candidates = []
    max_value = float('-inf')
    a1=0.5
    a2=-6
    a3=-2
    a4=1
    a5=3
    for lbl in dataset_label_distribution.keys():
        score = (
            a1*train_label_distribution.get(lbl, 0)
            +a2* neighbor_label_distribution.get(lbl, 0)
            +a3*feature_diffs[lbl]  # Penalize by average feature distance
        )
        print(lbl,score,)

        if score - max_value > 0.01:
            best_label_candidates = [lbl]
            max_value = score
        elif abs(score - max_value) <= 0.01:
            best_label_candidates.append(lbl)


    best_label = random.choice(best_label_candidates)
    y_pred[node] = best_label
    class_instances[best_label].add(node)  # Track newly labeled nodes
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    print(f"\n🔹 Labelling Node {node} ")
    # ✅ Print Prediction Outcome
    true_label = data.y[node].item()
    correct = "✅" if best_label == true_label else "❌"
    print(f"   -> Predicted Label: {best_label} | True Label: {true_label} {correct}")

    print(f"   - Total Neighbors: {len(neighbors_dict[node])}")
    print(f"   - Labeled Neighbors: {len(neighbor_labels)}")
    for lbl, count in label_counts.items():
        print(f"   - Class {lbl}: {count} neighbors")
        # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
        for neighbor in neighbors_dict[node]:
            if y_pred[neighbor] == -1:
                old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                if old_entry:
                    ordered_set.discard(old_entry)

                num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                total_neighbors = len(neighbors_dict[neighbor])

                new_weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                ordered_set.add((-new_weighted_score, neighbor))
    print("\n\n\n")

# ✅ Evaluate Accuracy
test_indices = [i for i in range(data.num_nodes) if i not in train_indices]
y_true = data.y[test_indices]
y_pred_test = y_pred[test_indices]

valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
if valid_idx:
    final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
    print(f"\n🎯 Label Propagation Accuracy: {final_accuracy:.4f}")
else:
    print("\n❌ No valid predictions were made!")



✅ Loaded  dataset with 183 nodes, 557 edges
   - Features: 1703
   - Number of Classes: 5
   - Number of Components: 1
55 1
24 1
11 1
11 1
17 1
3 -35.71553933368466
0 -34.46458492123984
4 -32.63789553758575
1 -33.952824736804494
2 -33.77085392649581

🔹 Labelling Node 21 
   -> Predicted Label: 4 | True Label: 2 ❌
   - Total Neighbors: 5
   - Labeled Neighbors: 5
   - Class 3: 3 neighbors
   - Class 0: 1 neighbors
   - Class 1: 1 neighbors




55 1
24 1
12 1
11 1
17 1
3 -17.204328319890713
0 -25.516642950414642
4 -20.957988692492975
1 -21.747741652698053
2 -23.982922484235065

🔹 Labelling Node 39 
   -> Predicted Label: 3 | True Label: 3 ✅
   - Total Neighbors: 1
   - Labeled Neighbors: 1
   - Class 0: 1 neighbors




56 1
24 1
12 1
11 1
17 1
3 -27.491918346746182
0 -32.80832996213339
4 -29.7259692680545
1 -29.95210452196075
2 -31.186766554669635

🔹 Labelling Node 51 
   -> Predicted Label: 3 | True Label: 0 ❌
   - Total Neighbors: 1
   - Labeled Neighbors: 1
   - Class 0: 1 neighbors


In [58]:
def predictclass(data,train_indices,test_indices,a1,a2,a3,a4,a5):
  data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)
  train_indices = list(train_indices)
  test_indices = list(test_indices)

# ✅ Remove duplicate edges
  data.edge_index = torch.unique(data.edge_index, dim=1)

  # ✅ Convert to NetworkX Graph to Find Components
  G = to_networkx(data, to_undirected=True)
  components = list(nx.connected_components(G))
  num_components = len(components)

#  print(f"\n✅ Loaded  dataset with {data.num_nodes} nodes, {data.edge_index.shape[1]} edges")
#  print(f"   - Features: {data.x.shape[1]}")
#  print(f"   - Number of Classes: {len(set(data.y.numpy()))}")
#  print(f"   - Number of Components: {num_components}")

  # ✅ Set Training Percentage
#  train_percentage = 0.6

  # ✅ Select Training Nodes
 # train_indices = []
  # for component in components:
  #     component = list(component)
  #     t = max(1, int(train_percentage * len(component)))
  #     sampled_nodes = np.random.choice(component, t, replace=False)
  #     train_indices.extend(sampled_nodes)

  np.random.shuffle(train_indices)

  # ✅ Initialize Prediction Labels
  y_pred = np.full(data.num_nodes, -1)
  for idx in train_indices:
      y_pred[idx] = data.y[idx].item()

  # ✅ Compute Label Distribution
  total_label_counts = Counter(data.y.numpy())
  num_labels = len(set(data.y.numpy()))
  alpha = 1  # Laplace smoothing

  dataset_label_distribution = {
      lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
      for lbl in total_label_counts.keys()
  }

  train_label_counts = Counter(y_pred[train_indices])

  train_label_distribution = {
      lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
      for lbl in dataset_label_distribution.keys()
  }

  # ✅ Precompute Neighbors for Each Node
  A = torch.zeros((data.num_nodes, data.num_nodes))
  A[data.edge_index[0], data.edge_index[1]] = 1
  A = A.numpy()
  neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

  # ✅ Track Instances Assigned to Each Label
  class_instances = defaultdict(set)
  for idx in train_indices:
      class_instances[data.y[idx].item()].add(idx)

  # ✅ Ordered Set for Managing Unlabeled Vertices
  ordered_set = SortedSet()
  train_labeled_nodes = set(train_indices)

  for node in range(data.num_nodes):
      if y_pred[node] == -1:  # Only process unlabeled nodes
          labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
          num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
          num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
          total_neighbors = len(neighbors_dict[node])

          if total_neighbors > 0:
              weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors
          else:
              weighted_score = 0

          ordered_set.add((-weighted_score, node))

  # ✅ Iteratively Label the Most Constrained Nodes
  while ordered_set:
  #    print("stop1")
      _, node = ordered_set.pop(0)

      if y_pred[node] != -1:
          print("something is wrong!!")

      # Get labeled neighbors
      neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
      label_counts = Counter(neighbor_labels)

      # 🚨 If this node has 0 labeled neighbors
      if not neighbor_labels:
          print(f"\n🚨 STRANGE: Node {node} was chosen, but it has 0 labeled neighbors!")
          print(ordered_set)
          break
   #   print("stop2")
      # Compute Neighbor Label Distribution
      neighbor_label_counts = Counter(neighbor_labels)
      total_labeled_neighbors = len(neighbor_labels)

      if total_labeled_neighbors > 0:
          neighbor_label_distribution = {
              lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
          }
      else:
          neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}
    #  print("stop3")
      # ✅ Compute Feature Distance to Each Class
      # feature_diffs = {}
      # for lbl in dataset_label_distribution.keys():
      #   if class_instances[lbl]:  # Only compute if we have instances of the label
      #       instance_features = data.x[list(class_instances[lbl])]
      #       node_feature = data.x[node].unsqueeze(0)

      #       # Compute distances for each instance
      #       distances = torch.norm(instance_features - node_feature, dim=1).tolist()

      #       # Apply weight based on whether the instance is from training or predicted labels
      #       weighted_distances = [
      #           (3 * dist) if instance in train_indices else dist  # Give 3x weight if in training set
      #           for instance, dist in zip(class_instances[lbl], distances)
      #       ]

      #       # Compute weighted average distance
      #       avg_distance = sum(weighted_distances) / len(weighted_distances)
      #       feature_diffs[lbl] = avg_distance
      #   else:
      #       feature_diffs[lbl] = 0  # No penalty if no instances exist

      feature_diffs = {}

      for lbl in dataset_label_distribution.keys():
      #  print("lbl=",lbl)
        if class_instances[lbl]:  # Only compute if we have instances of the label
          instance_features = data.x[list(class_instances[lbl])]
          node_feature = data.x[node].unsqueeze(0)

          if instance_features.numel() > 0:  # ✅ Ensure instance_features is not empty
          #  print(len(instance_features),len(node_feature))
            avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()

          else:
            avg_distance = 0  # ✅ Avoid empty slice issue
          #print("avg_distance=",avg_distance)
          feature_diffs[lbl] = avg_distance
        else:
          feature_diffs[lbl] = 0  # No penalty if no instances exist
   #     print("stop 4")

      # ✅ Select the Best Label Based on the Adjusted Score
      best_label_candidates = []
      max_value = float('-inf')

      for lbl in dataset_label_distribution.keys():
          score = (
              a1*train_label_distribution.get(lbl, 0)
              +a2* neighbor_label_distribution.get(lbl, 0)
              +a3*feature_diffs[lbl]  # Penalize by average feature distance
          )

          if score - max_value > 0.01:
              best_label_candidates = [lbl]
              max_value = score
          elif abs(score - max_value) <= 0.01:
              best_label_candidates.append(lbl)
   #   print("stop 5")
      best_label = random.choice(best_label_candidates)
      y_pred[node] = best_label
      class_instances[best_label].add(node)  # Track newly labeled nodes
      neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
      label_counts = Counter(neighbor_labels)

  #    print(f"\n🔹 Labelling Node {node} ")
      # ✅ Print Prediction Outcome
      true_label = data.y[node].item()
      correct = "✅" if best_label == true_label else "❌"
  #    print(f"   -> Predicted Label: {best_label} | True Label: {true_label} {correct}")

  #    print(f"   - Total Neighbors: {len(neighbors_dict[node])}")
  #    print(f"   - Labeled Neighbors: {len(neighbor_labels)}")
      for lbl, count in label_counts.items():
      #    print(f"   - Class {lbl}: {count} neighbors")
          # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
          for neighbor in neighbors_dict[node]:
              if y_pred[neighbor] == -1:
                  old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                  if old_entry:
                      ordered_set.discard(old_entry)

                  num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                  num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                  total_neighbors = len(neighbors_dict[neighbor])

                  new_weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                  ordered_set.add((-new_weighted_score, neighbor))
  #print("loop ended")
  # ✅ Evaluate Accuracy
  #test_indices = [i for i in range(data.num_nodes) if i not in train_indices]
  y_true = data.y[test_indices]
  y_pred_test = y_pred[test_indices]

  valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
  final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
  if valid_idx:
      final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
#      print(f"\n🎯 Label Propagation Accuracy: {final_accuracy:.4f}")
  else:
      print("\n❌ No valid predictions were made!")
  return y_pred_test,final_accuracy


In [79]:
#dataset = WebKB(root='data', name='Cornell')
dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
data = dataset[0]



# Count occurrences of each class label
class_counts = torch.bincount(data.y)

# Print the number of elements in each class
for class_id, count in enumerate(class_counts):
    print(f"Class {class_id}: {count} elements")
a1=0.5
a2=-6
a3=-2
a4=1
a5=3
indices = np.arange(data.num_nodes)  # NumPy array of indices [0, 1, 2, ..., num_nodes-1]

train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=42)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.2, random_state=42)
print(predictclass(data,train_idx,valid_idx,a1,a2,a3,a4,a5))

Processing...


Class 0: 33 elements
Class 1: 1 elements
Class 2: 18 elements
Class 3: 101 elements
Class 4: 30 elements
(array([3, 0, 1, 3, 4, 3, 3, 0, 3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 1, 3, 0, 3,
       4, 3, 3, 1, 3, 3, 0, 1, 3, 3, 4, 3, 4, 3, 2, 3, 3, 3, 0, 4, 1, 3,
       3, 3, 3, 2, 0, 3, 0, 1, 3, 3, 3, 4, 1, 2, 2, 4, 1, 4, 3, 3, 3]), 0.7692307692307693)


Done!


In [89]:
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import warnings
warnings.simplefilter("error", RuntimeWarning)  # Convert warnings to errors
#dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
dataset = WikipediaNetwork(root='data', name='squirrel')
dataset = WebKB(root='data', name='Wisconsin')


data = dataset[0]

num_nodes = data.x.shape[0]
indices = np.arange(num_nodes)

# Split into training (70%) and testing (30%) randomly
train_idx, test_idx = train_test_split(indices, test_size=0.4, random_state=42)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.5, random_state=42)

# Function to optimize (negate accuracy since we minimize)
# Function to optimize (maximize train accuracy by minimizing negative accuracy)

def random_search(model,data, train_idx,val_idx, num_samples=100):
    best_params = None
    best_score = float('-inf')

    for _ in range(num_samples):
        a1 = random.uniform(0, 10)     # a1 must be positive
        a2 = random.uniform(-10, -0.01)  # a2 must be negative
        a3 = random.uniform(-10, -0.01)  # a3 must be negative
        a5 = random.uniform(0, 10)     # a5 must be positive
        a4 = random.uniform(0,a5 )    # a4 must be positive and a4 >= a5

        params = [a1, a2, a3, a4, a5]


        _,score = model(data,train_idx,val_idx,params[0],params[1],params[2],params[3],params[4])
        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score



# Initial guesses for a1, a2, a3, a4, a5
initial_params = [0.5,-5,-2,1,3]

# Bounds for parameters
 # Adjust bounds as needed

# Optimize using scipy
result = random_search(predictclass,data,train_idx,val_idx,num_samples=10)[0]
#print(result)
# Get best parameters
best_a1, best_a2, best_a3, best_a4, best_a5 = result[0],result[1],result[2],result[3],result[4]

# Evaluate on test set
predicted_classes, final_accuracy = predictclass(data,train_idx,test_idx, best_a1, best_a2, best_a3, best_a4, best_a5)

# Print results
print("Optimal Parameters:", result)
print("Final Accuracy on Test Data:", final_accuracy)

Optimal Parameters: [4.487311987779512, -9.876398222599162, -5.558821427260015, 2.698110942973613, 4.334738343531611]
Final Accuracy on Test Data: 0.693069306930693
